In [1]:
import pandas as pd
import numpy as np

In [21]:
def integ(eps, t):
    if eps == 0:
        return t
    else:
        return (1/eps)*(1-np.exp(-eps*t))

def calc_prob(alpha, beta, gamma, t, mu):
    res = 0
    res += (1/64)*integ(1,t)
    res += (1/16)*gamma*np.exp(-t*mu)*integ(1-mu,t)
    res += (1/16)*(alpha+beta)*integ(1+mu,t)
    res += (1/4)*(alpha+beta)*gamma*np.exp(-t*mu)*integ(1,t)
    res += (1/4)*alpha*beta*integ(1+2*mu,t)
    res += alpha*beta*gamma*np.exp(-t*mu)*integ(1+mu,t)
    return res

def wrapper(a, b, c, d, t, mu):
    alpha = 3/4 if a==d else -1/4
    beta  = 3/4 if b==d else -1/4
    gamma = 3/4 if c==d else -1/4
    return (1/(1-np.exp(-t)))*calc_prob(alpha, beta, gamma, t, mu)

In [23]:
wrapper('A', 'C', 'A', 'A', 0.007, 0.2)

0.0001745310058324659

$P(b,c,d|a,\mu,t)=$

In [42]:
t = 0.5
theta = 2
mu = (4/3)*theta
arr = []
for a in ['A', 'C', 'T', 'G']:
    for b in ['A', 'C', 'T', 'G']:
        for c in ['A', 'C', 'T', 'G']:
            for d in ['A', 'C', 'T', 'G']:
                arr.append([a, b, c, d, wrapper(a, b, c, d, t, mu)])
df = pd.DataFrame(arr, columns = ['a', 'b', 'c', 'd', 'prob'])
df

,a,b,c,d,prob
0,A,A,A,A,0.287676
1,A,A,A,C,0.001102
2,A,A,A,T,0.001102
3,A,A,A,G,0.001102
4,A,A,C,A,0.069801
...,...,...,...,...,...
251,G,G,T,G,0.069801
252,G,G,G,A,0.001102
253,G,G,G,C,0.001102
254,G,G,G,T,0.001102


In [43]:
df.groupby(['a']).sum()

,prob
a,
A,1.0
C,1.0
G,1.0
T,1.0


$P(b,c|a,\mu,t)=$

In [44]:
df.groupby(['a', 'b', 'c']).sum()

prob
a b c          
A A A  0.290981
    C  0.082434
    G  0.082434
    T  0.082434
  C A  0.052239
...         ...
T G T  0.052239
  T A  0.082434
    C  0.082434
    G  0.082434
    T  0.290981

[64 rows x 1 columns]

In [60]:
def rate_mat(mu, t):
    return (1/4)+np.exp(-t*mu)*(1/4)*(np.full((4, 4), -1)+np.diag([4, 4, 4, 4]))

$P(a|A)=$

In [93]:
mu_A = 0.5
t_AB = 3
bases = {0:'A', 1:'C', 2:'T', 3:'G'}
arr2 = rate_mat(mu_A, t_AB)
df2 = pd.DataFrame(arr2).stack().reset_index()
df2['level_0'] = [bases[i] for i in df2['level_0']]
df2['level_1'] = [bases[i] for i in df2['level_1']]
df2.columns = ['A','a','Value']
df2

,A,a,Value
0,A,A,0.417348
1,A,C,0.194217
2,A,T,0.194217
3,A,G,0.194217
4,C,A,0.194217
5,C,C,0.417348
6,C,T,0.194217
7,C,G,0.194217
8,T,A,0.194217
9,T,C,0.194217


$P(a,A)=P(A)P(a|A)=(1/4)\cdot P(a|A)$

$P(b,B)=P(B)P(b|B)=(1/4)\cdot P(b|B)$

$P(c|A,B)=\sum_{d}P(c,d|A,B)=\sum_{d}\sum_{a}P(c,d|a,A,B)=\sum_{d}\sum_{a}\sum_{b}P(c,d|a,A,bB)$

$P(c|A,B)=\sum_{a}\sum_{b}P(c|a,A,b,B)=\sum_{a}\sum_{b}\frac{P(c,A,B|a,b)}{P(A,B)}$